In [0]:
import tensorflow as tf

In [2]:
print(tf.__version__)

1.14.0


In [0]:
from tensorflow.keras.datasets import cifar10
import sys
from tensorflow.keras import layers

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [0]:
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

In [0]:
class TFRecords:   
#   def __init__(self, out_filename, images, labels):
#     self.out_filename = out_filename
#     self.images = images
#     self.labels = labels

  def _int64_feature(self, value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

  def _bytes_feature(self, value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    
  def createDataRecord(self, out_filename, images, labels):

    writer = tf.io.TFRecordWriter(out_filename)

    for i in range(len(images)):
      feature = {
          'image_raw': self._bytes_feature(images[i].tostring()),
          'label': self._int64_feature(labels[i])
      }

      example = tf.train.Example(features=tf.train.Features(feature=feature))

      writer.write(example.SerializeToString())

    writer.close()
    sys.stdout.flush()

In [0]:
# Instantite the class

tfRecords = TFRecords()

In [0]:
tfRecords.createDataRecord('train.tfrecords', x_train, y_train)

In [0]:
tfRecords.createDataRecord('test.tfrecords', x_test, y_test)

In [12]:
!ls

drive  sample_data  test.tfrecords  train.tfrecords


In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    
    return output

In [17]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

W0902 03:58:11.561467 140448653805440 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [18]:
model = tf.keras.Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 12)   48          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 12)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 12)   48          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 12)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
def parser(record):
  keys_to_features = {
      'image_raw': tf.FixedLenFeature((), tf.string),
      'label': tf.FixedLenFeature((), tf.int64)
  }
  parsed = tf.parse_single_example(record, keys_to_features)
  image = tf.decode_raw(parsed['image_raw'], tf.uint8)
  image = tf.cast(image, tf.int32)
  image = tf.reshape(image, [32,32,3])
  label = tf.cast(parsed["label"], tf.int32)
  label = tf.one_hot(label, 10)
  return image, label

In [0]:
def get_dataset_from_TFRecord(filename):
  dataset = tf.data.TFRecordDataset(filenames='train.tfrecords')
  dataset = dataset.map(parser)
  return dataset    

In [0]:
dataset = get_dataset_from_TFRecord('train.tfrecords')
dataset = dataset.batch(512)
dataset = dataset.repeat()

In [0]:
val_dataset = get_dataset_from_TFRecord('test.tfrecords')

In [0]:
val_dataset = val_dataset.batch(512).repeat()

In [27]:
model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset,validation_steps=3)

W0902 03:58:16.188196 140448653805440 training_utils.py:1300] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/10
30/30 [==============================] - 27s 901ms/step - loss: 2.1380 - acc: 0.1970 - val_loss: 2.2591 - val_acc: 0.1393
Epoch 2/10
30/30 [==============================] - 12s 400ms/step - loss: 1.8200 - acc: 0.3078 - val_loss: 2.0229 - val_acc: 0.2383
Epoch 3/10
30/30 [==============================] - 12s 401ms/step - loss: 1.7021 - acc: 0.3611 - val_loss: 1.8746 - val_acc: 0.2982
Epoch 4/10
30/30 [==============================] - 15s 490ms/step - loss: 1.5957 - acc: 0.3998 - val_loss: 1.6421 - val_acc: 0.3978
Epoch 5/10
30/30 [==============================] - 12s 404ms/step - loss: 1.5250 - acc: 0.4261 - val_loss: 1.8263 - val_acc: 0.3639
Epoch 6/10
30/30 [==============================] - 12s 406ms/step - loss: 1.4734 - acc: 0.4513 - val_loss: 1.7014 - val_acc: 0.3861
Epoch 7/10
30/30 [==============================] - 12s 404ms/step - loss: 1.4036 - acc: 0.4752 - val_loss: 1.7565 - val_acc: 0.3848
Epoch 8/10
30/30 [==============================] - 12s 409ms/step - 